In [ ]:
# import os
# import sys
# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import pandas as pd
from quantfreedom.utils.caching import clear_cache
from quantfreedom.backtester.base.base import rsi_below_entries, plot_trades_all_info
from quantfreedom.backtester.nb.simulate import simulate_up_to_6
from quantfreedom.backtester.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)

pd.options.display.float_format = '{:,.2f}'.format

prices = pd.read_csv('30min.csv', index_col='time')

entries = rsi_below_entries(
    timeperiods=[15],
    below_ranges=[40],
    prices=prices.close
)


In [ ]:
final_array, order_records = simulate_up_to_6(
    open_prices=prices.open.values,
    high_prices=prices.high.values,
    low_prices=prices.low.values,
    close_prices=prices.close.values,
    entries=entries.values,
    equity=1000.,
    fee_pct=.06,
    mmr=.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=4,
    risk_rewards=[3,5,6],
    size_pct=1.,
    gains_pct_filter=40,
    sl_pcts=[2,3,4],
    # tsl_true_or_false=True,
    # tsl_pcts_init=np.arange(2,5,1),
    # tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    # tsl_trail_by_pct=np.arange(1,4,1),
    # tsl_when_pct_from_avg_entry=np.arange(1,4,1),
)


In [ ]:
df_or = pd.DataFrame(order_records)
for i in range(len(OrderType._fields)):
    df_or.replace({'order_type': {i: OrderType._fields[i]}}, inplace=True)
df_or.head(20)    

In [ ]:
df_cart = pd.DataFrame(final_array).dropna(axis='columns', thresh=1)
for i in range(len(SL_BE_or_Trail_BasedOn._fields)):
    df_cart.replace({'tsl_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
    df_cart.replace({'sl_to_be_based_on': {i: SL_BE_or_Trail_BasedOn._fields[i]}}, inplace=True)
df_cart.T

In [ ]:
plot_trades_all_info(
    open_prices=prices.open,
    high_prices=prices.high,
    low_prices=prices.low,
    close_prices=prices.close,
    order_records=order_records[:30],
)

In [ ]:
clear_cache()